In [11]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.rand(2, 20)

print(net(X))

tensor([[-0.0277, -0.2021,  0.1182,  0.0136, -0.0244, -0.0506,  0.0403,  0.0109,
         -0.2502, -0.0485],
        [ 0.0291, -0.1190,  0.0490, -0.0420,  0.0313,  0.0423,  0.0707,  0.0024,
         -0.1651,  0.0424]], grad_fn=<AddmmBackward0>)


In [12]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, x):
        return self.out(F.relu(self.hidden(x)))

In [14]:
net  = MLP()
print(net(X))

tensor([[-0.1515, -0.0734, -0.0552, -0.1136, -0.1307,  0.1394,  0.2825,  0.0846,
          0.2862, -0.1795],
        [-0.0513, -0.1965, -0.1761, -0.0359, -0.2581,  0.0568,  0.2420, -0.0529,
          0.2803, -0.1158]], grad_fn=<AddmmBackward0>)


In [15]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, x):
        for block in self.children():
            x = block(x)
        return x


In [17]:
net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)
print(net(X))

tensor([[-0.0053,  0.0406,  0.0556,  0.1047,  0.1005, -0.0892,  0.0493, -0.1996,
          0.0351, -0.1441],
        [-0.0653, -0.0378,  0.0875,  0.1515,  0.1838, -0.1095,  0.0607, -0.1725,
          0.1080, -0.2009]], grad_fn=<AddmmBackward0>)


In [19]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20),requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, x):
        x = self.linear(x)
        x = F.relu(torch.mm(x, self.rand_weight) + 1)
        x = self.linear(x)
        while x.abs().sum() > 1:
            x /= 2

        return x.sum()

In [21]:
net = FixedHiddenMLP()
print(net(X))

tensor(0.2750, grad_fn=<SumBackward0>)


In [23]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)

    def forward(self, x):
        return self.linear(self.net(x))

chimera = nn.Sequential(
    NestMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP()

)

chimera(X)



torch.Size([2, 20])
